# Notebook to take BLASTp _C. gigas_ output to GO slim

In [6]:
pwd

'/Users/graciecrandall/Documents/GitHub/paper-pacific.oyster-larvae/analyses'

In [10]:
wd="/Users/graciecrandall/Documents/GitHub/paper-pacific.oyster-larvae/analyses/0401-blastp/"

In [11]:
cd $wd

/Users/graciecrandall/Documents/GitHub/paper-pacific.oyster-larvae/analyses/0401-blastp


In [12]:
ls

0327-cgseedblast-sprot.tab


# Need three files

1) BLAST output format 6: https://github.com/grace-ac/paper-pacific.oyster-larvae/blob/master/analyses/0327-cgseedblast-sprot.tab        
2) Uniprot GO annotation file (340M) available here http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted       
3) GOslim file available here http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

In [13]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  2953k      0 --:--:-- --:--:-- --:--:-- 2997k


In [14]:
!curl -O  http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  3000k      0  0:01:56  0:01:56 --:--:-- 15.0M   0  2725k      0  0:02:07  0:01:41  0:00:26   496


In [17]:
#check that all three files are in directory
!ls

0327-cgseedblast-sprot.tab uniprot-SP-GO.sorted
GO-GOslim.sorted


In [18]:
#set blast output as variable
blastout="0327-cgseedblast-sprot.tab"

In [19]:
!head -2 $blastout

CHOYP_14332.1.2|m.5643	sp|Q2F637|1433Z_BOMMO	66.031	262	74	2	19	280	1	247	2.78e-119	344
CHOYP_14332.1.2|m.5644	sp|P62325|BTG1_MOUSE	47.205	161	80	2	1	156	11	171	2.22e-47	155


In [20]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [21]:
!head -2 _blast-sep.tab

CHOYP_14332.1.2	m.5643	sp	Q2F637	1433Z_BOMMO	66.031	262	74	2	19	280	1	247	2.78e-119	344
CHOYP_14332.1.2	m.5644	sp	P62325	BTG1_MOUSE	47.205	161	80	2	1	156	11	171	2.22e-47	155


In [22]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $4, $1, $14}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [23]:
!head -2 _blast-sort.tab

A0A0B5AC98	CHOYP_MPI1.1.1	1.61e-63
A0A0F7YYX3	CHOYP_LOC658640.1.2	3.61e-21


In [24]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [25]:
!head -2 _blast-annot.tab

A0A0B5AC98	CHOYP_MPI1.1.1	GO:0005179; GO:0005576; GO:0006006
A0A0F7YYX3	CHOYP_LOC658640.1.2	GO:0005576


# The following is a script modified from Sam

In [ ]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\t\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [ ]:
!head _blastout-GO-unfolded.tab

In [ ]:
#!sort -k3 _blast-GO-unfolded.tab > _blast-GO-unfolded.sorted

In [ ]:
#!tail _blast-GO-unfolded.sorted

In [ ]:
#!head GO-GOslim.sorted

In [ ]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
#!join -1 3 -2 1 -t $'\t' \
#_blast-GO-unfolded.sorted \
#GO-GOslim.sorted | head 

In [ ]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
#!join -1 3 -2 1 -t $'\t' \
#_blast-GO-unfolded.sorted \
#GO-GOslim.sorted \
#| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
#> Blastquery-GOslim.tab

In [ ]:
#!head Blastquery-GOslim.tab

In [ ]:
#!join -1 3 -2 1 -t $'\t' _blast-GO-unfolded.sorted GO-GOslim.sorted